<a href="https://colab.research.google.com/github/Nish077/BDI_QuestionWeightagePaper_dataset/blob/main/Artificial_dataset_generator_BDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
import os
import torch

In [5]:
# Mount Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/BDI_data/shap_values.npy'
shap_values = np.load(path, allow_pickle=True)
print(shap_values.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(832, 21)


In [10]:
print(type(shap_values))
print(shap_values.shape)
print(shap_values[0]) # Print the first element to check it is an array

<class 'numpy.ndarray'>
(832, 21)
[.values =
 -0.4165571166047084

 .base_values =
 10.464010975497747

 .data =
 0.0                 .values =
                     -0.2352411848312113

                     .base_values =
                     10.464010975497747

                     .data =
                     0.0                 .values =
                                         -0.15803378203370813

                                         .base_values =
                                         10.464010975497747

                                         .data =
                                         0.0
 .values =
 0.8542771969382739

 .base_values =
 10.464010975497747

 .data =
 2.0                .values =
                    0.36728108803537907

                    .base_values =
                    10.464010975497747

                    .data =
                    1.0                 .values =
                                        0.9247743144237028

                     

In [17]:
# Extract the '.values' attribute from each shap object
shap_values_list = [[shap_obj.values for shap_obj in row] for row in shap_values]

# # Convert the list of lists into a DataFrame
# shap_values_df = pd.DataFrame(shap_values_list)

# shap_values_df.head()


# Convert the list of lists into a NumPy array for easier calculations
shap_values_numerical = np.array(shap_values_list)
# Normalize SHAP values to create probabilities
shap_weights = np.abs(shap_values_numerical).mean(axis=0)  # Compute absolute mean
shap_probabilities = shap_weights / shap_weights.sum()  # Normalize

print("SHAP Probabilities:", shap_probabilities)

SHAP Probabilities: [0.06508402 0.03517823 0.02493531 0.04816431 0.03996063 0.04017465
 0.06391851 0.05031138 0.0158959  0.06627703 0.05574646 0.05007587
 0.0546118  0.03980241 0.04641947 0.04872454 0.04341931 0.05088109
 0.06229498 0.05619871 0.0419254 ]


In [18]:
# Define the number of rows and questions
num_rows = 347
num_questions = 21  # Since SHAP values were for 21 questions


In [21]:
response_probs = np.ones((num_questions, 4))
# Adjust response probabilities based on SHAP importance (higher SHAP → higher scores)
for i in range(num_questions):
    # Higher SHAP importance → more likely to have higher responses
    base_prob = shap_probabilities[i]
    response_probs[i] = np.array([1, 2, 3, 4]) * base_prob  # Scale with SHAP weight

    # Normalize to ensure they sum to 1
    response_probs[i] /= response_probs[i].sum()

# Generate synthetic responses for each question using adjusted probabilities
synthetic_data = np.array([
    [np.random.choice(range(4), p=response_probs[q]) for q in range(num_questions)]
    for _ in range(num_rows)
])
ages = np.random.randint(18, 41, size=num_rows)
genders = np.random.choice([0, 1], size=num_rows)
total_scores = synthetic_data.sum(axis=1)


In [22]:

# Create DataFrame
column_names = [f"Q{i+1}" for i in range(num_questions)] + ["Age", "Gender", "Total_Score"]
df_synthetic = pd.DataFrame(np.column_stack((synthetic_data, ages, genders, total_scores)), columns=column_names)

# Save dataset to CSV
df_synthetic.to_csv("synthetic_bdi_dataset.csv", index=False)

print("Synthetic dataset generated and saved!")
df_synthetic.head()  # Show preview of the dataset

Synthetic dataset generated and saved!


,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Age,Gender,Total_Score
0,2,0,1,3,2,2,3,0,3,2,...,3,2,2,1,1,2,1,25,1,36
1,3,2,2,3,3,2,3,1,3,2,...,2,3,3,1,2,3,2,18,1,49
2,3,3,3,1,3,3,3,2,2,3,...,3,1,1,2,3,2,3,18,0,50
3,2,3,1,3,2,1,2,3,2,1,...,1,1,3,1,1,2,1,32,1,37
4,2,2,1,2,1,2,3,2,1,1,...,0,2,3,3,2,1,2,22,0,34
